In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import cartopy as cp
from matplotlib.patches import FancyArrowPatch
from scipy.ndimage import map_coordinates
import metpy.calc as mpcalc
from geographiclib.geodesic import Geodesic
from utils_datetime import *
from utils_filter import *

In [ ]:
grid_outlooks = xr.open_dataset('data/outlooks/grid_outlooks.nc')
grid_outlooks = grid_outlooks.sel(time = grid_outlooks['time'] >= '200203300000')

In [ ]:
# creates new displacements dataset by keeping data (get rid of shifts and div) 
# but changing x and y coords so that (0, 0) is at max or weighted center of outlook (or pph?). Since highest risk is seen as "center" of what will happen, even if weaker storms are focused to one direction
# then calculate things like shifts of quadrents

# note: grids will not quite be aligned when each is centered over a different point, but probably close enough


# finding x and y to make center for each date
grouped = grid_outlooks['prob'].groupby('time')

# Step 1: Find all points with the maximum prob for each day and compute mean coordinates
def find_mean_coords(group):
    max_prob = group.max()  # Maximum value in the group
    if max_prob == 0:
        mean_x = group['x'].mean().item()
        mean_y = group['y'].mean().item()
    else:
        # Select all points with prob == max_prob
        max_points = group.where(group == max_prob, drop=True)
        # Compute the mean of x and y
        mean_x = max_points['x'].mean().item()
        mean_y = max_points['y'].mean().item()
    # Round to nearest integers
    nearest_x = round(mean_x)
    nearest_y = round(mean_y)
    # Return as a Dataset
    return xr.Dataset({'nearest_x': nearest_x, 'nearest_y': nearest_y})

# Apply the function to each group
center_coords = grouped.map(find_mean_coords)
center_coords

del grid_outlooks, grouped

<xarray.Dataset>
Dimensions:    (time: 7947)
Coordinates:
  * time       (time) object '200203300000' '200203310000' ... '202312310000'
Data variables:
    nearest_x  (time) int32 50 65 55 56 67 55 70 44 ... 46 46 46 46 46 46 46 46
    nearest_y  (time) int32 18 22 32 24 24 21 14 22 ... 32 32 32 32 32 32 32 32

In [19]:
# Extract nearest_x and nearest_y
nearest_x = center_coords['nearest_x']
nearest_y = center_coords['nearest_y']


# Re-center displacements
def recenter_displacements(displacements, nearest_x, nearest_y):
    # Shift x and y coordinates based on nearest_x, nearest_y for each time
    new_x = displacements['x'] - nearest_x
    new_y = displacements['y'] - nearest_y
    
    # Assign shifted coordinates
    displacements = displacements.assign_coords({
        'x': new_x,
        'y': new_y
    })
    
    # Update lat and lon to depend on time, x, and y
    
    
    return displacements
# Initialize an empty list to store the recentered displacements



for dataset_location in ['data/pph/labelled_pph', 'data/displacement/displacements', 'data/outlooks/grid_outlooks', ]:
    print(dataset_location)
    recentered_list = []
    ds = xr.open_dataset(dataset_location + '.nc')
    ds = ds.sel(time = ds['time'] >= '200203300000')

    # Loop over each time step and apply recenter_displacements
    for t in range(len(ds.time)):
        # Get the current time slice of displacements
        displacement_slice = ds.isel(time=t)

        # Get the corresponding nearest_x and nearest_y for this time step
        nearest_x_t = nearest_x.isel(time=t)
        nearest_y_t = nearest_y.isel(time=t)

        

        # Apply the recentering function to the current time slice
        recentered_t = recenter_displacements(displacement_slice, nearest_x_t, nearest_y_t)

        # expanding needs to be done differently

        # Append the recentered displacements for this time slice
        recentered_list.append(recentered_t)
    print('combining')
    # Combine the recentered displacements back into a single xarray object
    recentered = xr.concat(recentered_list, dim='time')
    # Ensure proper ordering of dimensions
    #recentered = recentered.transpose('time', 'y', 'x', 'hazard')
    recentered.to_netcdf(dataset_location + '_recentered.nc')
    del recentered, recentered_list


data/displacement/displacements
combining
data/outlooks/grid_outlooks
combining


In [66]:
# open pph, outlook, and displacements
#test_time = ['201104270000', '201905310000']
# do something like how shifts and divergence are calculated but in each quadrent
pph_recentered = xr.open_dataset('data/pph/labelled_pph_recentered.nc')
outlooks_recentered = xr.open_dataset('data/outlooks/grid_outlooks_recentered.nc')
displacements_recentered = xr.open_dataset('data/displacement/displacements_recentered.nc')


In [67]:
hazard_types= ['Wind', 'Hail', 'Tornado', 'All Hazard']

pph_key_dict = {
    'Wind': 'p_perfect_wind',
    'Hail': 'p_perfect_hail',
    'Tornado': 'p_perfect_tor',
    'All Hazard': 'p_perfect_totalsvr'
}

outlook_key_dict = {
    'Wind': 'Day 1 Wind',
    'Hail': 'Day 1 Hail',
    'Tornado': 'Day 1 Tornado',
    'All Hazard': 'Day 1'
}

displacements_recentered = displacements_recentered.assign(e_shift_n = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(n_shift_n = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(total_div_n = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))

displacements_recentered = displacements_recentered.assign(e_shift_e = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(n_shift_e = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(total_div_e = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))

displacements_recentered = displacements_recentered.assign(e_shift_s = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(n_shift_s = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(total_div_s = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))

displacements_recentered = displacements_recentered.assign(e_shift_w = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(n_shift_w = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))
displacements_recentered = displacements_recentered.assign(total_div_w = (('time', 'hazard'), np.full((len(displacements_recentered['time']), len(hazard_types)), 0.0)))

for side in ['n', 'e', 's', 'w']:
    print(side)
    for hazard in hazard_types:

        print(hazard)

        e_shifts = []
        n_shifts = []
        total_divs = []

        hazard_dataset = displacements_recentered.sel(hazard = hazard)
        for date in displacements_recentered['time']:

            weights = outlooks_recentered.sel(time = date, outlook = outlook_key_dict[hazard])['prob']
            if side == 'n':
                mask = weights.y >= 0
            elif side == 'e':
                mask = weights.x >= 0
            elif side == 's':
                mask = weights.y <= 0
            elif side == 'w':
                mask = weights.x <= 0
            weights = weights.where(mask, 0).fillna(0).data
            
            if weights.max() == 0: # no outlook, so weight at pph
                weights = pph_recentered.sel(time = date)[pph_key_dict[hazard]].where(mask, 0).fillna(0).data
            if weights.max() == 0:
                weights = None
            hazard_time_dataset = hazard_dataset.sel(time = date)
            e_shift = np.average(hazard_time_dataset['e_flow'].fillna(0), weights = weights)
            n_shift = np.average(hazard_time_dataset['n_flow'].fillna(0), weights = weights)
            div = np.gradient(hazard_time_dataset['x_flow'].fillna(0))[1] + np.gradient(hazard_time_dataset['y_flow'].fillna(0))[0]
            total_div = np.average(div, weights = weights)

            displacements_recentered['e_shift_' + side].loc[dict(time = date, hazard = hazard)] = e_shift
            displacements_recentered['n_shift_' + side].loc[dict(time = date, hazard = hazard)] = n_shift
            displacements_recentered['total_div_' + side].loc[dict(time = date, hazard = hazard)] = total_div

n
Wind
Hail
Tornado
All Hazard
e
Wind
Hail
Tornado
All Hazard
s
Wind
Hail
Tornado
All Hazard
w
Wind
Hail
Tornado
All Hazard


In [ ]:
displacements_recentered.to_netcdf('data/displacement/displacements_recentered_test.nc')

In [70]:
# exploring the data
displacements_recentered.sel(time = '201104270000')

<xarray.Dataset>
Dimensions:      (x: 152, y: 99, hazard: 4)
Coordinates:
  * x            (x) float64 -80.0 -79.0 -78.0 -77.0 ... 68.0 69.0 70.0 71.0
  * y            (y) float64 -44.0 -43.0 -42.0 -41.0 ... 51.0 52.0 53.0 54.0
  * hazard       (hazard) object 'Wind' 'Hail' 'Tornado' 'All Hazard'
    time         <U12 '201104270000'
Data variables: (12/23)
    lat          (y, x) float64 ...
    lon          (y, x) float64 ...
    x_flow       (y, x, hazard) float64 ...
    y_flow       (y, x, hazard) float64 ...
    end_lon      (y, x, hazard) float64 ...
    end_lat      (y, x, hazard) float64 ...
    ...           ...
    e_shift_s    (hazard) float64 3.116e+05 1.5e+05 1.165e+05 1.225e+05
    n_shift_s    (hazard) float64 2.461e+05 1.374e+05 1.121e+04 7.547e+04
    total_div_s  (hazard) float64 0.5059 0.6156 0.7457 0.636
    e_shift_w    (hazard) float64 3.067e+05 -1.715e+04 3.985e+03 1.077e+04
    n_shift_w    (hazard) float64 2.091e+05 8.915e+04 -4.24e+04 -2.274e+04
    total_div_w  (hazard) float64 0.7096 0.8611 0.8107 0.4939
Attributes:
    description:  Displacements from gridded day 1 probabilistic outlooks to ...
    grid:         80-km NCEP 211